<a href="https://colab.research.google.com/github/jonnythebard/tf2_workbook/blob/master/explore_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.layers import Input, Dense, SimpleRNN, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.x  # Colab only.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
2.0.0


In [0]:
# create the data

# N = number of samples
# T = sequence length
# D = number of input features
# M = number of hidden units
# K = number of output units

N = 1
T = 10
D = 3
K = 3
X = np.random.randn(N, T, K)

In [0]:
# make an rnn
M = 5
i = Input(shape=(T, D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)

model = Model(i, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 3)]           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 45        
_________________________________________________________________
dense (Dense)                (None, 3)                 18        
Total params: 63
Trainable params: 63
Non-trainable params: 0
_________________________________________________________________


In [0]:
# see how weights look like at rnn layer
model.layers[1].get_weights()

[array([[ 0.3223037 , -0.4845738 ,  0.6530085 , -0.37492505, -0.55669856],
        [ 0.6429983 ,  0.4588154 , -0.28153634,  0.29270667, -0.6110543 ],
        [ 0.68853706,  0.44633645,  0.1228323 ,  0.1438412 , -0.51814234]],
       dtype=float32),
 array([[-0.46037364, -0.3305035 , -0.02389528,  0.79504895, -0.21482556],
        [ 0.1860227 , -0.21288691, -0.95778286,  0.00449002,  0.05202463],
        [-0.39886698, -0.7018375 ,  0.06717698, -0.56401783, -0.16031356],
        [-0.5370635 ,  0.574246  , -0.2609683 , -0.21936333, -0.51534426],
        [-0.55310017,  0.15202764, -0.09728207, -0.04050634,  0.8123197 ]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [0]:
# check shapes of each layer
# first output is input > hidden
# second output is hidden > hidden
# third output is bias term (vector of lengh M)
Wx, Wh, bh = model.layers[1].get_weights()
Wo, bo = model.layers[2].get_weights()

# Wx - input to hidden weight
# Wh - hidden to hidden weight
# bh - hidden bias
# Wo - hidden to output weight
# bo - output bias
print("in rnn layer..")
print("Wx shape:", Wx.shape)
print("Wh shape:", Wh.shape)
print("bh shape:", bh.shape)
print("in output layer..")
print("Wo shape:", Wo.shape)
print("bo shape:", bo.shape)

in rnn layer..
Wx shape: (3, 5)
Wh shape: (5, 5)
bh shape: (5,)
in output layer..
Wo shape: (5, 3)
bo shape: (3,)


formula to get Yhat is as follow:

![naive rnn fomula](https://drive.google.com/uc?id=1fHtVRtgycSdUM8UxYclyGzvtsoO0kDf3)

In [0]:
# lets code the equation above to mimic model.predict

# don't have to code activation function in output layer though
# because I didn't specify it when building the model

h_last = np.zeros(M) # initial hidden state
x = X[0]
Yhats = []

for t in range(T):
  h = np.tanh(x[t].dot(Wx) + h_last.dot(Wh) + bh)
  y = h.dot(Wo) + bo
  Yhats.append(y)

  h_last = h

print(Yhats[-1])
print(model.predict(X)[0])

[ 0.56733789 -0.3363996  -1.09093379]
[ 0.5673379  -0.33639956 -1.0909338 ]
